In [32]:
from utils.t5 import *
import warnings
import gc
torch.cuda.empty_cache()
pl.seed_everything(42)
warnings.filterwarnings("ignore")
gc.collect()
torch.cuda.empty_cache()

Global seed set to 42


In [33]:
input_model_name = "d-t5-t5-base-grammar-correction_grammar_added"
input_data_name = "preprocessed.csv"  

input_model_dir = "./Data/Model/"
input_data_dir = "./Data/Preprocessed/"
output_dir = "./Data/Generation/"
source_column = "outline"           
generated_column = "automatic_text"       
model_type = "t5"

input_data_path = "{}{}".format(input_data_dir, input_data_name)
new_data_path = "{}generated_{}.csv".format(output_dir,input_model_name)
input_model_path = "{}{}".format(input_model_dir,input_model_name)

In [35]:
data = pd.read_csv(input_data_path)

In [36]:
train, dev_test = train_test_split(data, test_size = 0.3, random_state = 42)
dev, test = train_test_split(dev_test, test_size = 0.5, random_state = 42)

In [37]:
model = SimpleT5()

In [38]:
input_model_path

'./Data/Model/d-t5-t5-base-grammar-correction_grammar_added'

In [39]:
model.load_model(model_type, input_model_path, use_gpu = True)

In [40]:
input_data = ("grammar: " + test[source_column]).to_list()
#input_data = test[source_column].to_list()
print(len(input_data))
automatic_lst = model.predict(input_data, batch_size = 15) # 30 15 4

43


  0%|          | 0/3 [00:00<?, ?it/s]

In [41]:
new_data_path

'./Data/Generation/generated_d-t5-t5-base-grammar-correction_grammar_added.csv'

In [42]:
test[generated_column] = automatic_lst
print(len(test))
test.to_csv(new_data_path, index=False)

43
